## Using Tensorflow for Classification: ##

Tensorflow is Deep Learning Framework from Google. It has become quite popular in recent years and help in putting Neural Network Architecture together

This kernel is about doing a classification task using Tensorflow. This example uses a DNNClassifier for performing Binary Classification. The data used is **Breast Cancer data from Wisconsin.** This has 32 Columns and 569 rows.

The classifier used is DNNClassifier.
 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas.api.types as ptypes
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df_wisconsin = pd.read_csv("BreastCancerDetection.csv")
df_wisconsin.columns = df_wisconsin.columns.str.replace('\s+', '_')  # Replacing column names by _ whereever space id found
len(df_wisconsin.columns)

33

**Determining numeric columns from the dataset.**
Following is a strategy to find out which are numeric columns. This is done because, later on when building feature columns, this information will be required.

In [3]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_wisconsin_numeric = df_wisconsin.select_dtypes(include=numerics) # exclude is another keyword.
# Looking at the columns of the dataset
len(df_wisconsin_numeric.columns)

32

We observe that there is difference of only one column, which is the **diagnosis** column. This will also be the response or target column. Out of these 32 columns, *id* and *Unnamed:_32* will be ignored.

Numeric columns are for further consideration are:

'radius_mean',
'texture_mean', 
'perimeter_mean',
'area_mean',
'smoothness_mean',
'compactness_mean', 
'concavity_mean',
'concave points_mean', 
'symmetry_mean',
'fractal_dimension_mean',
'radius_se',
 'texture_se', 
'perimeter_se', 
 'area_se',
 'smoothness_se',
 'compactness_se',
  'concavity_se',
  'concave points_se',
  'symmetry_se',
  'fractal_dimension_se', 
  'radius_worst',
  'texture_worst',
  'perimeter_worst',
  'area_worst', 
   'smoothness_worst',
  'compactness_worst', 
   'concavity_worst', 
   'concave points_worst',
   'symmetry_worst', 
   'fractal_dimension_worst'
   
   
   Above mentioned numeric columns will be normalized.

In [4]:
normalize_columns = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave_points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave_points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave_points_worst',
       'symmetry_worst', 'fractal_dimension_worst']

We observe that Diagnosis is the column which we need to predict on unseen data. 
Data seems to be pretty well balanced and possible values are **Benign** and **Malignant**.

In [5]:
x_values = df_wisconsin.drop(['diagnosis','id','Unnamed:_32'],axis=1) # Getting Predictors
y_val = df_wisconsin['diagnosis'] # getting response

In [6]:
x_values.head() # Examining x_values

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
y_val.value_counts() # Checking if the classes are balanced. Seems pretty good.

B    357
M    212
Name: diagnosis, dtype: int64

In [8]:
# Converting Labels to integer form. 'B' and 'M; are represented as 0,1. Since Tensorflow does not accept categorical variables in text form, we are converting to integers.
def label_numeric(label):
    if (label == 'B'):
        return(0)
    else:
        return(1)

In [9]:
y_val_numeric =y_val.apply(label_numeric)
y_val_numeric.value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

A quick check above confirms that things are happening as per the expectation.

## Performing Training and Testing split ##
Preparing data for model building and checking. 10 percent of data is kept aside for validation purposes. We will also be performing scaling of data using sklearn's *MinMaxScaler*.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x_values,y_val_numeric,test_size=0.1,random_state=1234)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)
#X_train.head()
# Have a look at the before and after scaling step to understand what transformation has been done to the data.

In [11]:
# Similarly, scaling is performed on test data as well.
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)
X_test.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
531,0.221922,0.411438,0.217124,0.115673,0.531878,0.257345,0.098407,0.107207,0.403535,0.308551,...,0.192814,0.440486,0.182230,0.089805,0.553589,0.261092,0.220288,0.279038,0.323477,0.226026
166,0.180747,-0.028596,0.172621,0.090817,0.470403,0.130060,0.059302,0.084394,0.162121,0.295703,...,0.130559,-0.012686,0.115793,0.056233,0.478307,0.095478,0.083626,0.158179,0.103489,0.143972
485,0.258839,0.257362,0.269919,0.141336,0.461714,0.451065,0.361762,0.240855,0.516162,0.490522,...,0.208111,0.230499,0.236117,0.097179,0.305950,0.367523,0.391054,0.461168,0.328405,0.317198
66,0.117564,0.453692,0.112777,0.053404,0.562290,0.199815,0.050890,0.074751,0.331818,0.400800,...,0.088225,0.514710,0.082773,0.035760,0.552268,0.134965,0.075176,0.223952,0.258821,0.243146
220,0.315633,0.118651,0.304678,0.180445,0.476051,0.231936,0.091097,0.127386,0.151515,0.283909,...,0.263607,0.104184,0.245530,0.128048,0.395760,0.213552,0.140495,0.276838,0.160654,0.210809


### Building feature columns for Tensorflow framework ###
Tensorflow expects feature columns to be built using its api calls. Following is a lengthy way of creating them. Later on, an efficient mechanism is also used. Following is retained as it makes the understanding easier.

In [12]:
# Now we have 30 features which will be built in tensorflow framewor. Tensorflow requires these features to be defined as feature columns.
fc_radius_mean = tf.feature_column.numeric_column('radius_mean')
fc_texture_mean = tf.feature_column.numeric_column('texture_mean')
fc_perimeter_mean = tf.feature_column.numeric_column('perimeter_mean')
fc_area_mean = tf.feature_column.numeric_column('area_mean')
fc_smoothness_mean = tf.feature_column.numeric_column('smoothness_mean')
fc_compactness_mean = tf.feature_column.numeric_column('compactness_mean')
fc_concavity_mean = tf.feature_column.numeric_column('concavity_mean')
fc_concave_points_mean = tf.feature_column.numeric_column('concave points_mean')
fc_symmetry_mean = tf.feature_column.numeric_column('symmetry_mean')
fc_fractal_dimension_mean = tf.feature_column.numeric_column('fractal_dimension_mean')
fc_radius_se = tf.feature_column.numeric_column('radius_se')
fc_texture_se = tf.feature_column.numeric_column('texture_se')
fc_perimeter_se = tf.feature_column.numeric_column('perimeter_se')
fc_area_se = tf.feature_column.numeric_column('area_se')
fc_smoothness_se = tf.feature_column.numeric_column('smoothness_se')
fc_compactness_se = tf.feature_column.numeric_column('compactness_se')
fc_concavity_se = tf.feature_column.numeric_column('concavity_se')
fc_concave_points_se = tf.feature_column.numeric_column('concave points_se')
fc_symmetry_se = tf.feature_column.numeric_column('symmetry_se')
fc_fractal_dimension_se = tf.feature_column.numeric_column('fractal_dimension_se')
fc_radius_worst = tf.feature_column.numeric_column('radius_worst')
fc_texture_worst = tf.feature_column.numeric_column('texture_worst')
fc_perimeter_worst = tf.feature_column.numeric_column('perimeter_worst')
fc_area_worst = tf.feature_column.numeric_column('area_worst')
fc_smoothness_worst = tf.feature_column.numeric_column('smoothness_worst')
fc_compactness_worst = tf.feature_column.numeric_column('compactness_worst')
fc_concavity_worst = tf.feature_column.numeric_column('concavity_worst')
fc_concave_points_worst = tf.feature_column.numeric_column('concave points_worst')
fc_symmetry_worst = tf.feature_column.numeric_column('symmetry_worst')
fc_fractal_dimension_worst = tf.feature_column.numeric_column('fractal_dimension_worst')
#feat_cols = [fc_radius_mean, ..., fc_concave_points_worst, fc_symmetry_worst]


In [13]:
# df = X_train
# for col in df.columns:
#     if ptypes.is_string_dtype(df[col]): #is_string_dtype is pandas function
#         print(col)
#     else :
#         print("else ",type(df[col]))
#print(df.columns)
print(tf.feature_column.numeric_column('radius_mean'))

NumericColumn(key='radius_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [14]:
# Efficient way of building feature columns. 
# Please notice that Categorical Columns and Numerical Columns are treated differently.
feat_cols = []
df = X_train
for col in df.columns:
    if ptypes.is_string_dtype(df[col]): #is_string_dtype is pandas function
        feat_cols.append(tf.feature_column.categorical_column_with_hash_bucket(col, 
        hash_bucket_size= len(df[col].unique())))
        
    elif ptypes.is_numeric_dtype(df[col]): #is_numeric_dtype is pandas function
        feat_cols.append(tf.feature_column.numeric_column(col))
        

Having a look at the Feature Columns as below:

In [15]:
print(feat_cols)

[NumericColumn(key='radius_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='texture_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='perimeter_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='area_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='smoothness_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='compactness_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='concavity_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='concave_points_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='symmetry_mean', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='fractal_dimension_me

### Choosing Classification model, input function and training steps. ###
*  Classification model -  tf.estimator.DNNClassifier is chosen as the model for training the data.
*  Input function - Since we have loaded data using Pandas input function, we will use pandas_input_fn. Tensorflow also provides numpy based input function.
*  epocs and steps - Since these terms are used frequently, it is important to understand the difference between them.
    Following explanation is taken from Stackoverflow:
    
>     In the neural network terminology:
>         one epoch = one forward pass and one backward pass of all the training examples
>         batch size = the number of training examples in one forward/backward pass. The higher the batch size, the more memory space you'll need.
>         number of iterations = number of passes, each pass using [batch size] number of examples. To be clear, one pass = one forward pass + one backward pass (we do not count the forward pass and backward pass as two different passes).

In [16]:
classifier = tf.estimator.DNNClassifier(
        feature_columns=feat_cols,
        # Two hidden layers of 20 nodes each.
        hidden_units=[20, 20],
        # The model must choose between 2 classes.
        n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/61/41z9g_ts09v_8vy_wnjpt8nc0000gq/T/tmp0n89los8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb204db2550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=500, shuffle=True)

AttributeError: module 'tensorflow_core.estimator' has no attribute 'inputs'

### Now it is time to train the Classifier ###

In [ ]:
classifier.train(input_fn=input_func, steps=10000)

Creating an input function for prediction, will be applied on test data. Please note that shuffle parameter is set to False here.

In [ ]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False) # Shuffle should be set as false as we are interested in comparing with actual results.

In [ ]:
# Prediction is done here now.
predictions = list(classifier.predict(input_fn=pred_fn))
predictions[0]

Since prediction API outputs a number of information, we are taking the required stuff here which is class_ids.

In [ ]:
final_preds = []
for pred in predictions:
    #info = "{} {} {}".format(pred['class_ids'][0], pred['probabilities'][0] , pred['probabilities'][1])
    final_preds.append(pred['class_ids'][0])
    #final_preds.append(info)

### Evaluating model performance ###
Once we have prediction data available with us, we can now compare with the 10% data we have kept aside. As we have kept aside the unseen data, we can be reasonably confident that our model has not overfit. This is important step in machine learning to not to use evaluation data for training a model.
We have used classification report, accuracy and confusion matrix to evaluate the model performance.

In [ ]:
print(classification_report(y_test,final_preds))

In [ ]:
print(confusion_matrix(y_test,final_preds))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,final_preds, normalize=True, sample_weight=None)

We see that out of 57 observations, only 1 is misclassified. This concludes this article of using Tensorflow for classification. ** Happy Model Building! #**

*Note: This has been a good experience using Tensorflow for NN. Hope you liked!*